pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [8]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = 'jdonadelli'
password = open('passwd.txt', 'r').read()

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [4]:
# tv.search_symbol('PETR4','BMFBOVESPA')
# df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
# df.head(3)

In [5]:
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 10)
df

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-08-09 16:10:00,BMFBOVESPA:PETR4,36.44,36.54,36.42,36.52,378800.0
2024-08-09 16:15:00,BMFBOVESPA:PETR4,36.54,36.56,36.50,36.54,405900.0
2024-08-09 16:20:00,BMFBOVESPA:PETR4,36.54,36.60,36.54,36.58,582200.0
2024-08-09 16:25:00,BMFBOVESPA:PETR4,36.58,36.63,36.57,36.58,439600.0
2024-08-09 16:30:00,BMFBOVESPA:PETR4,36.58,36.68,36.58,36.62,784900.0
2024-08-09 16:35:00,BMFBOVESPA:PETR4,36.63,36.64,36.54,36.56,435900.0
2024-08-09 16:40:00,BMFBOVESPA:PETR4,36.55,36.57,36.50,36.51,397300.0
2024-08-09 16:45:00,BMFBOVESPA:PETR4,36.50,36.54,36.47,36.51,2528300.0
2024-08-09 16:50:00,BMFBOVESPA:PETR4,36.51,36.53,36.44,36.48,598400.0


In [6]:
url  = {'ibra50':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBXLDia_15-12-23.csv',
        'ibra100':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBXXDia_15-12-23.csv',
        'idiv':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IDIVDia_15-12-23.csv',
        'ibov':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBOVDia_15-12-23.csv',
        'small':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SMLLDia_15-12-23.csv'
}

indice = pd.read_csv(url['small'] ,
                     sep=';' , decimal=',' , thousands='.' , header=1 , skiprows=0 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
for idx,row in indice.iterrows():
    i=row['Código']
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [ ]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]